In [27]:
import os
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

"""
本代码通过技术科提供基准介对应的图号,并将该图号BOM录入SageX3-ERP中，
对应ERP的采购入库材料价格,获得该BOM的材料及其物料价格。
1。 输入
1.1 BOM清单
1.2 物料替代清单
1.3 最新物料入库价格表
1.4 读取所有图号的BOM集合
"""

# 一. 读取所有数据输入源
path = r"J:\python_code\cost_estimate\src\核价工具11.0\\"
file_material = r"最新入库价(7).xlsx"
file_material_matched = r"对照表.xlsx"
file_BOM_drawing = r"所有图号清单.xlsx"
file_BOM_Replace = r"材料替代清单.xlsx"
file_BOM_All = r"所有图号BOM清单.xlsx"

# 1. 读入最新物料价格

material = pd.read_excel(path+file_material,
                            sheet_name="Sheet1"
                            # ,engine="openpyxl"
                            )
# print(material)

# 2. 读入对照表,作用是区别主材与其他材料
material_matched = pd.read_excel(path+file_material_matched,
                   sheet_name="Sheet2"
                #    ,engine="openpyxl"
                   )
material_matched.drop_duplicates(subset=["替代物料识别辅助列"], keep='first', inplace=True)

# 3 读入技术科提供的图号清单
BOM_drawing = pd.read_excel(path+file_BOM_drawing,
                            sheet_name="Sheet1"
                            # ,engine="openpyxl"
                            )

# 4 读取物料替代清单<材料替代清单.xlsx>
BOM_Replace = pd.read_excel(path+file_BOM_Replace,
                            sheet_name="Sheet1"
                            # ,engine="openpyxl"
                            )

# 1.3 读取PG库中所有图号的BOM清单并改列名
BOM_All = pd.read_excel(path+file_BOM_All,
                            sheet_name="Sheet1"
                            # ,engine="openpyxl"
                            )

In [28]:
BOM_All = BOM_All.iloc[:,1:]

In [29]:
col_1 = BOM_All[1:].columns.tolist()

In [30]:
print(col_1)

['zstofcy_0', 'itmdes1_0', 'itmdes2_0', 'cpnitmref_0', 'sub_itmdes1_0', 'sub_itmdes2_0', 'likqty_0', 'sub_stu_0', 'credat_0', 'stu_0', 'puustucoe_0', 'puu_0']


In [31]:
col_2 = ["地点","图号编码","图号名称","图号","材料编码","材料名称","材料代号","库存数量","库存单位","创建日期","单位","采购数量","采购单位"]
# 组成字典,用于改列名
col_name =  dict(zip(col_1, col_2))

In [26]:
print(col_name)

{'图号编码': '图号编码', '地点': '地点', '图号名称': '图号名称', '图号': '图号', '材料编码': '材料编码', '材料名称': '材料名称', '材料代号': '材料代号', '库存数量': '库存数量', '库存单位': '库存单位', '创建日期': '创建日期', '单位': '单位'}


In [10]:
BOM_All.rename(columns= col_name,inplace=True)

In [19]:
tmp = BOM_All.copy()

tmp["校准前"] = tmp["材料名称"].astype(str) + tmp["材料代号"].astype(str)
# set(tmp["材料名称"])